In [ ]:
! pip install pandas;
! pip install pgmpy;

In [18]:
import pandas as pd;
from pgmpy.models import BayesianNetwork

Quali = "Qualifikation"
S = "Schnitt"
BL = "Bundesland"
M = "Mathe"
P = "Physik"
D = "Deutsch"
ST = "Schultyp"
OLTM = "OLT-Mathe"
OLTD = "OLT-Deutsch"
SFT = "Studierfähigkeitstest"
A = "Alter"
Geschl = "Geschlecht"
JEDE = "Jahreseinkommen der Eltern"
SB = "Staatsbürgerschaft"
SG = "Studiengang"
Absch= "Abschluss"

df = pd.read_csv('p001_1.csv', sep=';',dtype={
    Quali: 'string',
    S: 'string',
    BL: 'string',
    M: 'string',
    D: 'string',
    P: 'string',
    ST: "string",
    OLTM: "string",
    OLTD: "string",
    SFT: "string",
    A: "string",
    Geschl: "string",
    JEDE: "string",
    SB: "string",
    SG: "string",
    Absch: "string"
})
df = df.replace({'n.a.': "-1"})
df = df.replace({ "keine": "-1"})
df[S] = df[S].str.replace(",",".").astype('float') 
df[M] = df[M].str.replace(",",".").astype('float') 
df[D] = df[D].str.replace(",",".").astype('float') 
df[P] = df[P].str.replace(",",".").astype('float') 

df[OLTM] = df[OLTM].astype('int') 
df[OLTD] = df[OLTD].astype('int') 
df[SFT] = df[SFT].astype('int') 
df[A] = df[A].astype('int') 
df[JEDE] = df[JEDE].astype('int') 
display(df)
G = BayesianNetwork()
G.add_nodes_from(df.columns)

G.add_edges_from([(ST, SG),(Quali, S),(S, M),(S,D),(S,P),(P, OLTM),(M, OLTM),(D, OLTD),(OLTM, Absch),(OLTD, Absch),(BL, JEDE),(JEDE, SFT),(JEDE, SG),(SG,Absch),(A, SFT),(Geschl, SFT),(SB, SFT)])
display(df.dtypes)
display(G.nodes)
display(G.edges)
G.fit(df)

values = pd.DataFrame({BL:["Baden-Württemberg"],
                        ST:["Allgemeinbildendes Gymnasium"],
                        M:["2,3"],
                        D:["2,1"],
                        P:["2,2"],
                        Quali:["Abitur"],
                        SFT:["685"]})
predict_data = values
y_pred = G.predict(predict_data)
print(y_pred)


,Qualifikation,Schnitt,Bundesland,Mathe,Physik,Deutsch,Schultyp,OLT-Mathe,OLT-Deutsch,Studierfähigkeitstest,Alter,Geschlecht,Jahreseinkommen der Eltern,Staatsbürgerschaft,Studiengang,Abschluss
0,Abitur,2.7,Baden-Württemberg,2.3,2.2,2.1,Allgemeinbildendes Gymnasium,63,62,685,19,m,47000,deutsch,Maschinenbau,3
1,Meister,1.6,Nordrhein-Westfalen,-1.0,-1.0,-1.0,-1,36,59,-1,25,m,87000,deutsch,Maschinenbau,abgebrochen
2,Abitur,1.8,Baden-Württemberg,1.0,1.1,1.2,Wirtschaftsgymnasium,96,94,-1,18,w,115000,deutsch,Soziale Arbeit,"1,1"
3,Abitur,1.1,Baden-Württemberg,2.5,2.7,1.9,Technisches Gymnasium,70,76,-1,17,m,115000,deutsch,Elektrotechnik,"2,6"
4,Abitur,1.4,Bayern,2.0,2.1,1.4,Wirtschaftsgymnasium,65,82,-1,17,m,90000,deutsch,Elektrotechnik,"2,3"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Abitur,1.4,Baden-Württemberg,2.4,2.5,2.5,Allgemeinbildendes Gymnasium,60,65,-1,19,w,107000,deutsch,Elektrotechnik,"3,1"
96,Abitur,2.6,Baden-Württemberg,1.1,1.1,1.2,Technisches Gymnasium,100,94,-1,20,m,80000,deutsch,Wirtschaftswissenschaften,"1,5"
97,Abitur,1.6,Baden-Württemberg,2.0,2.2,1.7,Technisches Gymnasium,76,86,-1,19,m,208000,deutsch,Maschinenbau,"2,4"
98,Abitur,2.6,Bayern,1.5,1.4,1.1,Allgemeinbildendes Gymnasium,80,86,-1,18,m,110000,deutsch,Maschinenbau,"1,8"


Qualifikation                  string
Schnitt                       float64
Bundesland                     string
Mathe                         float64
Physik                        float64
Deutsch                       float64
Schultyp                       string
OLT-Mathe                       int64
OLT-Deutsch                     int64
Studierfähigkeitstest           int64
Alter                           int64
Geschlecht                     string
Jahreseinkommen der Eltern      int64
Staatsbürgerschaft             string
Studiengang                    string
Abschluss                      string
dtype: object

NodeView(('Qualifikation', 'Schnitt', 'Bundesland', 'Mathe', 'Physik', 'Deutsch', 'Schultyp', 'OLT-Mathe', 'OLT-Deutsch', 'Studierfähigkeitstest', 'Alter', 'Geschlecht', 'Jahreseinkommen der Eltern', 'Staatsbürgerschaft', 'Studiengang', 'Abschluss'))

OutEdgeView([('Qualifikation', 'Schnitt'), ('Schnitt', 'Mathe'), ('Schnitt', 'Deutsch'), ('Schnitt', 'Physik'), ('Bundesland', 'Jahreseinkommen der Eltern'), ('Mathe', 'OLT-Mathe'), ('Physik', 'OLT-Mathe'), ('Deutsch', 'OLT-Deutsch'), ('Schultyp', 'Studiengang'), ('OLT-Mathe', 'Abschluss'), ('OLT-Deutsch', 'Abschluss'), ('Alter', 'Studierfähigkeitstest'), ('Geschlecht', 'Studierfähigkeitstest'), ('Jahreseinkommen der Eltern', 'Studierfähigkeitstest'), ('Jahreseinkommen der Eltern', 'Studiengang'), ('Staatsbürgerschaft', 'Studierfähigkeitstest'), ('Studiengang', 'Abschluss')])

100%|██████████| 1/1 [00:00<00:00, 839.53it/s]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices